# Rice Leafs 4 Classify Fasiai
<br/>
This noteboook refered to the following notebook.<br/>

'First Steps: Road to the Top, Part 1' by JEREMY HOWARD<br/>

https://www.kaggle.com/code/jhoward/first-steps-road-to-the-top-part-1

In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
from fastkaggle import *

## Set up

In [ ]:
comp = 'riceleafs/RiceLeafs'
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')
display(path)

In [ ]:
from fastai.vision.all import *
set_seed(42)
path.ls()

# Show the data

In [ ]:
trn_path = path/'train'
files = get_image_files(trn_path)

In [ ]:
img = PILImage.create(files[0])
print(img.size)
img.to_thumb(128)

In [ ]:
from fastcore.parallel import *
def f(o): return PILImage.create(o).size
sizes = parallel(f, files, n_workers=8)
pd.Series(sizes).value_counts()

# Train Data - ImageDataLoaders.from_folder

In [ ]:
#### train and valid data in train folder
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42,
    item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75))
dls.show_batch(max_n=6)

#### train and valid folders separated
dls = ImageDataLoaders.from_folder(path, train='train', valid='val', seed=42,
    item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75))
dls.show_batch(max_n=6)

# Model - 'resnet50d'

In [ ]:
learn = vision_learner(dls, 'resnet50d', metrics=error_rate, path='.').to_fp16() #'resnet26d'

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
learn.fine_tune(30,0.01)

# Predict using Test Data

In [ ]:
tst_files = get_image_files(path/'validation').sorted()
tst_dl = dls.test_dl(tst_files)
print(tst_files[0:3])

In [ ]:
probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)

In [ ]:
print(idxs)
print(dls.vocab)

In [ ]:
mapping = dict(enumerate(dls.vocab))
print(mapping)
reverse_mapping=dict(zip(dls.vocab,list(range(len(dls.vocab)))))
print(reverse_mapping)

In [ ]:
results = pd.Series(idxs.numpy(),name="idxs").map(mapping)
display(results)

In [ ]:
testY=[]
for item in tst_files:
    testY+=[reverse_mapping[item.parent.name]]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(testY,idxs,target_names=dls.vocab,digits=4))

#### classification_report

* 'resnet26d', learn.fine_tune(30,0.01), f1-score 0.3234 
* 'resnet26d', learn.fine_tune(60,0.01), f1-score 0.3025
* 'densenet201', learn.fine_tune(30,0.01), f1-score 0.2832
* 'tv_resnet101', learn.fine_tune(30,0.01), f1-score 0.2727
* 'efficientnet_b4', learn.fine_tune(30,0.01), f1-score 0.2250 
* 'mobilenetv2_140', learn.fine_tune(30,0.01), f1-score 0.1773
* 'resnet50d', learn.fine_tune(30,0.01), f1-score 